In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data
import matplotlib.pyplot as plt
import os
import tifffile as tiff
from glob import glob
from torchvision import datasets, transforms
from torchvision.transforms import ToPILImage
from torchvision.io import read_image
from PIL import Image
from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_error
from skimage import io
import cv2
from sklearn.linear_model import RidgeClassifier
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import make_scorer, roc_auc_score, accuracy_score, f1_score, balanced_accuracy_score, precision_score, recall_score, roc_curve
from sklearn.model_selection import GridSearchCV
from sklearn.base import BaseEstimator
from sklearn.utils import compute_sample_weight
from optuna.pruners import SuccessiveHalvingPruner
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
from optuna.visualization import plot_optimization_history
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import torchvision.models as models













if torch.cuda.is_available():
    print("Using GPU...")
    dev = "cuda:0"
else:
    print("Using CPU...")
    dev = "cpu"
    
    
device = torch.device(dev)






#Set the random seed for reproducibility 
torch.manual_seed(2020) 



class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            # Transform the 28 by 28 image to an embedded code size of 30
            nn.Linear(38400, 30),
            nn.ReLU()
        )
        self.decoder = nn.Sequential(
            nn.Linear(30, 38400),
            nn.Sigmoid()  #to range [0, 1]
        )

    def forward(self, x):
        # x will be a (batch_size,1,28,28) tensor when using MNIST
        # so we will reshape it to a (batch_size, 28*28) flat tensor.
        # After it has been decoded we will reshape back to the image shape.
        x = self.encoder(x.view(-1, 38400))
        x = self.decoder(x)
        return x.view(-1, 1, 160, 240)




class ConvAutoencoder(nn.Module):
    def __init__(self):
        super(ConvAutoencoder, self).__init__()
        self.encoder = nn.Sequential(
            # 1 input image channel, 16 output channel, 3x3 square convolution
            nn.Conv2d(1, 16, 3, stride=2, padding=1),
            nn.ReLU(),  # activation function
            nn.Conv2d(16, 32, kernel_size=3, stride=2, padding=1),  # conv layer
            nn.ReLU(), # activation function 
            nn.Conv2d(32, 64, kernel_size=7, stride=1, padding=0) # conv layer
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(64, 32, kernel_size=7, stride=1, padding=0), # conv transpose layer
            nn.ReLU(), # activation function
            nn.ConvTranspose2d(32, 16, 3, stride=2, padding=1, output_padding=1), # conv transpose layer
            nn.ReLU(), # activation function
            nn.ConvTranspose2d(16, 1, 3, stride=2, padding=1, output_padding=1),
            nn.Sigmoid()  #to range [0, 1]
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x







class ConvAutoencoder2D(nn.Module):
    def __init__(self):
        super(ConvAutoencoder2D, self).__init__()
        self.encoder = nn.Sequential(
            # 1 input image channel, 16 output channel, 3x3 square convolution
            nn.Conv2d(1, 16, 3, stride=2, padding=1),
            nn.ReLU(),  # activation function
            nn.Conv2d(16, 32, kernel_size=3, stride=2, padding=1),  # conv layer
            nn.ReLU(), # activation function 
            nn.Conv2d(32, 64, kernel_size=7, stride=1, padding=0), # conv layer
            nn.ReLU(),
            #nn.Linear( 64*1*1, 2)
            nn.Linear( 64*1*1, 2)
        )
        self.decoder = nn.Sequential(
            
            nn.ConvTranspose2d(64, 32, kernel_size=7, stride=1, padding=0), # conv transpose layer
            nn.ReLU(), # activation function
            nn.ConvTranspose2d(32, 16, 3, stride=2, padding=1, output_padding=1), # conv transpose layer
            nn.ReLU(), # activation function
            nn.ConvTranspose2d(16, 1, 3, stride=2, padding=1, output_padding=1),
            nn.Sigmoid()  #to range [0, 1]
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

In [ ]:
imgA = './UCSD_Anomaly_Dataset.v1p2/UCSDped1/Test_all/Test017_141.tif'
imgB = './UCSD_Anomaly_Dataset.v1p2/UCSDped1/Test_all/Test017_143.tif'


imlist = [imgA,imgB]

w,h=Image.open(imlist[0]).size
N=len(imlist)

arr=np.zeros((h,w),float)


for im in imlist:
    imarr=np.array(Image.open(im),dtype=float)
    arr=arr+imarr/N


arr=np.array(np.round(arr),dtype=np.uint8)

out=Image.fromarray(arr)
#out.save("017_142.tif")
#out.show()


path = './UCSD_Anomaly_Dataset.v1p2/UCSDped1/Test_all'
imageList = [os.path.join(path, file) for file in os.listdir(path)]
print(imageList[0])

w,h=Image.open(imageList[0]).size
N=len(imageList)

avgArr=np.zeros((h,w),float)

for im in imageList:
    imageArr=np.array(Image.open(im),dtype=float)
    avgArr=avgArr+imageArr/N
    
avgArr=np.array(np.round(avgArr),dtype=np.uint8)
    
avg=Image.fromarray(avgArr)
#avg.save("Test_avg.tif")
avg.show()

    


In [ ]:


num_epochs = 30

"""
class AnomalyDataset(data.Dataset):
    
    def __init__(self, img_dir, transform=None):
        self.img_dir = img_dir
        self.folder_names = os.listdir(self.img_dir)
        self.transform = transform
        
    def __len__(self):
        return len(self.folder_names)
        
    
    def __getitem__(self, index):
        #img_path = self.img_dir[index]
        #image = Image.open(img_path)
        #avg = Image.open('Test_avg.tif')
        #image1 = np.asarray(image)
        #image2 = np.asarray(avg)
        #image = image1 - image2
        #image = Image.fromarray(image)
        #image.show()
        #if self.transform:
        #    image = self.transform(image)
        
        folder_path = os.path.join(self.img_dir, self.folder_names[index])
        image_filenames = os.listdir(folder_path)
        
        images = []
        for img in image_filenames:
            if img.endswith('.tif'):
                image_path = os.path.join(folder_path, img)
                image = Image.open(image_path)
            
                if self.transform:
                    image = self.transform(image)
                images.append(image)
            
        return images
      
      
"""   

        
path = './UCSD_Anomaly_Dataset.v1p2/UCSDped1/All_data'
data_transforms = transforms.Compose([transforms.Resize((160, 240)), transforms.ToTensor()])

#dataset = AnomalyDataset(path, data_transforms)



#dataloader = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=True)

    
    

root_dir = './UCSD_Anomaly_Dataset.v1p2/UCSDped1/All_data'
num_folders = 70
num_images_per_folder = 200
image_height = 160
image_width = 240

dataset = np.zeros((num_folders, num_images_per_folder, image_height, image_width))

for i in range(num_folders):
    folder_name = str(i+1).zfill(3)
    folder_path = os.path.join(root_dir, folder_name)

    for j in range(num_images_per_folder):
        image_name = str(j+1).zfill(3) + '.tif'
        image_path = os.path.join(folder_path, image_name)

        image = Image.open(image_path)
        image = image.resize((240, 160))
        
        image = np.array(image)
        dataset[i, j, :, :] = image
    
    
    
print(dataset.shape)
dataset_norm = dataset/255.0

np.save('UCSD_Ped1', dataset_norm)

#dataset_tensor = torch.tensor(dataset)
#print(dataset_tensor.shape)



                
                
            
            
            
            

#image_path_train = './UCSD_Anomaly_Dataset.v1p2/UCSDped1/Train_all'
#image_paths_train = glob(image_path_train + '/*.tif')
#data_transforms = transforms.Compose([transforms.Resize((160, 240)), transforms.ToTensor()])



#image_path_test = './UCSD_Anomaly_Dataset.v1p2/UCSDped1/Test_all'
#image_paths_test = glob(image_path_test + '/*.tif')

#dataset_train = AnomalyDataset(image_paths_train, data_transforms)
#dataset_train2 = AnomalyDataset(image_paths_test, data_transforms)




#dataset = UCSDAnomalyDataset('./UCSD_Anomaly_Dataset.v1p2/UCSDped1/Train', time_stride=1)
#data_train = data.DataLoader(dataset, batch_size=32, shuffle=True)



In [ ]:
label_start = 34
labels = np.zeros((num_folders, num_images_per_folder, 1))
"""    
labels = np.zeros(7200, dtype=np.int8)
import csv
with open('labels.txt', 'r') as file:
    for i, line in enumerate(file):
        if("," in line):
            #part1 = line.rstrip().split(',')[0]
            #part2 = line.rstrip().split(',')[1]
            start1 = int(line.rstrip().split(',')[0].split(':')[0])-1+(i*200)
            end1 = int(line.rstrip().split(',')[0].split(':')[1])+1+(i*200)
            
            labels[start1:end1] = 1
            
            start2 = int(line.rstrip().split(',')[1].split(':')[0])-1+(i*200)
            end2 = int(line.rstrip().split(',')[1].split(':')[1].replace(';', ''))+1+(i*200)
            
            labels[start2:end2] = 1
            
            
        else:
            start3 = int(line.rstrip().split(';')[0].split(':')[0])-1+(i*200)
            end3 = int(line.rstrip().split(';')[0].split(':')[1])+1+(i*200)
            #s = slice(*map(int, frames.split(':')))
            #print(s)
            labels[start3:end3] = 1
            
"""
            
            
            
with open("labels.txt", "r") as f:
    for i, line in enumerate(f):
        parts = line.strip().split(";")
        if ("," in line):
            indices = parts[0].strip().split(",")
            for index in indices:
                start, end = index.strip().split(":")
                start = int(start) - 1
                end = int(end) - 1
                
                labels[i+label_start, start:end, 0] = 1
        else:
            start, end = parts[0].strip().split(":")
            start = int(start) - 1
            end = int(end) - 1
            
            labels[i+label_start, start:end, 0] = 1
                
                
                
print(labels.shape)

In [ ]:

X_train, X_test, y_train, y_test = train_test_split(dataset_norm, labels, test_size=0.2, random_state=42)
print(y_test.shape)


#img = Image.fromarray(np.uint8(X_train[0][0]))

#plt.imshow(img)
#plt.show()





#cae.to(device)
    




def do_pca(n_components, train_data, test_data):
    pca = PCA(n_components)
    
    #train_data = train_data.numpy()
    num_videos, num_frames, height, width = train_data.shape
    data_train = train_data.reshape(num_videos*num_frames, height*width)
    print(data_train.shape)
    
    transformed_train_images = pca.fit_transform(data_train)
    reconstructed_train_images = pca.inverse_transform(transformed_train_images)
    print(reconstructed_train_images.shape)
    
    #test_data = test_data.numpy()
    num_videos, num_frames, height, width = test_data.shape   
    data_test = test_data.reshape(num_videos*num_frames, height*width)
    transformed_test_images = pca.transform(data_test)
    reconstructed_test_images = pca.inverse_transform(transformed_test_images)
    

    #print(np.cumsum(pca.explained_variance_ratio_))

    plt.plot(np.arange(1,n_components+1), pca.explained_variance_ratio_)
    # Plotting using a log scale might show more information
    #plt.yscale('log')
    plt.ylabel('Explained Variance Ratio')
    plt.xlabel('Number of Components')
    plt.show()

    plt.plot(np.arange(1,n_components+1), np.cumsum(pca.explained_variance_ratio_))
    plt.ylabel('Cumulative Explained Variance Ratio')
    plt.xlabel('Number of Components')
    plt.show()
    



    reconstruction_error_train = mean_squared_error(data_train, reconstructed_train_images)
    print('Train Reconstruction error is ', reconstruction_error_train)
    print("\n\n")

    plt.figure(figsize=(16, 4))
    for i in range(8):
        # Top row: show original faces
        plt.subplot(2,8,i+1)
        plt.xticks([])
        plt.yticks([])
        plt.imshow(data_train[i].reshape(160,240), cmap='Greys_r')
        # Bottom row: show reconstructions
        plt.subplot(2,8, 8+i+1)
        plt.xticks([])
        plt.yticks([])
        plt.imshow(reconstructed_train_images[i].reshape(160,240), cmap='Greys_r')
    plt.show()
    
    reconstruction_error_test = mean_squared_error(data_test, reconstructed_test_images)
    print('Test Reconstruction error is ', reconstruction_error_test)
    print("\n\n")

    plt.figure(figsize=(16, 4))
    for i in range(8):
        # Top row: show original faces
        plt.subplot(2,8,i+1)
        plt.xticks([])
        plt.yticks([])
        plt.imshow(data_test[i].reshape(160,240), cmap='Greys_r')
        # Bottom row: show reconstructions
        plt.subplot(2,8, 8+i+1)
        plt.xticks([])
        plt.yticks([])
        plt.imshow(reconstructed_test_images[i].reshape(160,240), cmap='Greys_r')
    plt.show()
    
    return transformed_train_images, transformed_test_images

    
    
print("==========PCA TRAIN/TEST==========")
#pca_100_train, pca_100_test = do_pca(100, X_train, X_test)
#pca_200_train, pca_200_test = do_pca(200, X_train, X_test)
#pca_400_train, pca_400_test = do_pca(400, X_train, X_test)
#pca_600_train, pca_600_test = do_pca(600, X_train, X_test)


In [ ]:
ae = Autoencoder()
cae = ConvAutoencoder()
cae2D = ConvAutoencoder2D()

#print(ae)
#print("==============Standard Autoencoder===========")
#outputs = train_AE(ae, dataset, max_epochs=30, print_steps=30)
#print("\n\n")

"""
numImgs=12;
for k in range(0, len(outputs), 9):
    plt.figure(figsize=(numImgs, 2))
    imgs = outputs[k][1].numpy()    
    recon = outputs[k][2].numpy()
    print('Epoch:', k+1)
    for i, item in enumerate(imgs):
        if i >= numImgs: break
        plt.subplot(2, numImgs, i+1)
        plt.xticks([])
        plt.yticks([])
        plt.imshow(item[0])
        
    for i, item in enumerate(recon):
        if i >= numImgs: break
        plt.subplot(2, numImgs, numImgs+i+1)
        plt.xticks([])
        plt.yticks([])
        plt.imshow(item[0])
"""


print("==============ConvAutoencoder TRAIN==============")
outputs = train_AE(cae, X_train, max_epochs=30, print_steps=30)
print("\n\n")



        
        
print("\n\n")
print("==============ConvAutoencoder TEST==============")
outputs = train_AE(cae, X_test, max_epochs=30, print_steps=30)
print("\n\n")



        
numOut = len(outputs)-1
numImgs=12;
for k in range(0, len(outputs), 9):
    plt.figure(figsize=(numImgs, 2))
    imgs = outputs[k][1].cpu().numpy()    
    recon = outputs[k][2].cpu().numpy()
    print('Epoch:', k+1)
    for i, item in enumerate(imgs):
        if i >= numImgs: break
        plt.subplot(2, numImgs, i+1)
        plt.xticks([])
        plt.yticks([])
        plt.imshow(item[0])
        
    for i, item in enumerate(recon):
        if i >= numImgs: break
        plt.subplot(2, numImgs, numImgs+i+1)
        plt.xticks([])
        plt.yticks([])
        plt.imshow(item[0])
        

plt.figure(figsize=(16, 4))
imgs = outputs[numOut][1].cpu().numpy()    
recon = outputs[numOut][2].cpu().numpy()

for i in range(8):
    # Top row: show original faces
    plt.subplot(2,8,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.imshow(imgs[i][0].reshape(160,240), cmap='Greys_r')
    # Bottom row: show reconstructions
    plt.subplot(2,8, 8+i+1)
    plt.xticks([])
    plt.yticks([])
    plt.imshow(recon[i][0].reshape(160,240), cmap='Greys_r')
plt.show()
        

In [ ]:
def train_AE(model, dataset, max_epochs=20, print_steps=5):
    #Training (optimisation) parameters
    batch_size=64
    learning_rate=1e-3

    #Choose mean square error loss
    criterion = nn.MSELoss() 

    #Choose the Adam optimiser
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-5)

    #Specify how the data will be loaded in batches (with random shuffling)
    train_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)

    #Storage
    outputs = []
    
    model.to(device)
    #train_loader = train_loader.cuda()

    #Start training
    for epoch in range(max_epochs):
        for img in train_loader:
            img = img.to(device)
            recon = model(img.float())
            #print(recon.shape)
            loss = criterion(recon, img.to(device))

            loss.backward()
            optimizer.step()  
            optimizer.zero_grad()
          
        #if ((epoch % print_steps) == 0) or (epoch +1 == max_epochs):
        print('Epoch:{}, Loss:{:.4f}'.format(epoch+1, loss.item()))
        outputs.append((epoch, img.detach(), recon.detach()),)

    return outputs

In [ ]:
#print(outputs[29][1].cpu().numpy().shape)
#print(outputs[29][])
#print(pca_400.shape)

#linearModel = LinearRegression()

#linearOutputs = trainLinearModel(linearModel, pca_400, labels, 50, 50)

V, F = y_train.shape[:2]
labels_train = np.reshape(y_train, (V*F, 1))

V, F = y_test.shape[:2]
labels_test = np.reshape(y_test, (V*F, 1))
labels_test_log = labels_test.ravel()
labels_train_log = labels_train.ravel()

linearPCA = RidgeClassifier().fit(pca_400_train, labels_train_log)

y_pred = linearPCA.predict(pca_400_test)
accuracy = balanced_accuracy_score(labels_test_log, y_pred)
print("Accuracy of Ridge Classifier: ", accuracy)
auc = roc_auc_score(labels_test_log, y_pred)
print("AUC score for Ridge Classifier: ", auc)
print("F1 score for Ridge Classfier", f1_score(labels_test_log, y_pred))

#linearPCA.score(pca_400_test, labels_test)


clf = LogisticRegression().fit(pca_400_train, labels_train_log)
pred = clf.predict(pca_400_test)
acc = balanced_accuracy_score(labels_test_log, pred)

print("Accuracy of Logistic regression: ", acc)
auc_log = roc_auc_score(labels_test_log, pred)
print("AUC score for Logistic regression: ", auc_log)
print("F1 score for Logistic regression", f1_score(labels_test_log, pred))



    


#linearAE = RidgeClassifier().fit(outputs[29][1].cpu().numpy().flatten().reshape(-1, 1), labels)
#linearAE.score(outputs[29][1].cpu().numpy(), labels)

In [ ]:


def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0)

def sigmoid(x):
    return 1 / (1 + np.exp(-x))
    

class ESN(BaseEstimator):
    def __init__(self, input_size, reservoir_size, output_size, leaking_rate=0.95, spectral_radius=1, input_scaling=1, threshold=0.5, sparsity=0.1, ridge_alpha=0.5, random_seed=75):
        np.random.seed(random_seed)
        self.input_size = input_size
        self.reservoir_size = reservoir_size
        self.output_size = output_size

        self.W_in = np.random.randn(reservoir_size, input_size)
        self.W_res = np.random.randn(reservoir_size, reservoir_size)
        #self.W_out = np.random.randn(output_size, reservoir_size)
        
        self.leaking_rate = leaking_rate
        self.spectral_radius = spectral_radius
        self.input_scaling = input_scaling
        self.threshold = threshold
        self.sparsity = sparsity
        self.ridge_alpha = ridge_alpha
        
        e, v = np.linalg.eig(self.W_res)
        max_abs = np.max(np.abs(e))
        self.W_res = self.W_res/max_abs
        M = (np.random.uniform(size=(reservoir_size, reservoir_size)) < self.sparsity) * 1.0
        self.W_res *= M
        
        self.h0 = np.zeros((reservoir_size, 1))
        
        #print("W_res: ", self.W_res.)
        #print(max_abs)

        

        
        
        
        
    def update(self, x):
        h = self.h0
        T = x.shape[0]
        states = np.zeros((T, self.reservoir_size))

        for t in range(T):
            #print(x[t].reshape(-1, 1).shape)
            h = self.leaking_rate * np.tanh(self.spectral_radius * self.W_res @ h + self.input_scaling * self.W_in @ 
                                                                                 x[t].reshape(-1, 1)) + (1-self.leaking_rate)*h
            
            states[t] = h.flatten()
            
        #print(states.shape)
        
        return states
    

    def fit(self, X, y):
        V, F, D = X.shape
        self.W_out = np.random.randn(self.output_size, self.reservoir_size)
        states = np.zeros((V, F, self.reservoir_size))
        
        for v in range(V):
            states[v] = self.update(X[v])
        
        states = states.reshape(V*F, self.reservoir_size)
        
            
        
        ridge = Ridge(alpha=self.ridge_alpha)
        weights = compute_sample_weight(class_weight='balanced', y=y.flatten())
        ridge.fit(states, y.flatten(), sample_weight=weights)
        self.W_out = ridge.coef_
            
            #print(W_out.shape)
            #self.W_out = W_out
            
        #print(self.W_out.shape)
            
        #return self.W_out
            
            
    
    
    def predict(self, X):
        V, F, D = X.shape
        pred = np.zeros((V, F, 1))
        probs = np.zeros((V, F, 1))
        
        print("W_Res:")
        print(self.W_res * self.spectral_radius)
        print("=======================================================")
        print("W_in")
        print(self.W_in * self.input_scaling)
        for v in range(V):
            states = self.update(X[v])
            
            
            for f in range(F):
                state = states[f]
                output = np.dot(state.reshape(-1, 1).T, self.W_out.reshape(self.reservoir_size, -1))
                #y_prob = sigmoid(output)
                pred[v][f] = (output >= self.threshold).astype(int)
                #print(pred[v][f].shape)
                probs[v][f] =  output
        
        return pred, probs
            
            
            #print(states.shape)
            #y = states @ self.W_out[v].T
            #print(y.shape)
            #print(self.W_out.shape)
            #print("W_out", self.W_out[v].shape)
            #print("states", states.shape)
            
            #w = self.W_out[v].reshape(self.reservoir_size, 1)
            #y_pred[v] = sigmoid(np.sum(w * states.reshape((1, F, self.reservoir_size, 1)), axis=2))

            #print("New W: ", w.shape)
            #w = self.W_out[v].reshape((1, F, self.reservoir_size, 1))
            #s = states.reshape((1, F, self.reservoir_size, 1))
            #print("w", w.shape)
            #print("s", s.shape)
            
            #y_prob = np.zeros((F, 1))
            #y_pred = np.dot(self.W_out, states.T)
            #y_prob = softmax(y_pred)
            #y_pred_labels = np.argmax(y_prob, axis=1)
            #for t in range(F):
            #    y_pred = np.dot(self.W_out, states[t])
            #    y_prob[t] = sigmoid(y_pred)
                
            #y_prob = sigmoid(y_pred)
            #print(y_prob.T.shape)
            #probs[v] = y_prob.reshape(-1, 1)
            #print(y_pred[2])
            #print("y_pred", y_pred.reshape(-1, 1).shape)
            
            #y_pred = softmax(self.W_out[v] @ states.T)
            #print(y_pred.shape)
            #pred[v] = (y_prob.reshape(-1, 1) >= self.threshold).astype(int)
            
            
        #return pred, probs
        
        
        
        
        
        

        
        

    





In [ ]:
VF, D = pca_200_train.shape
V = int(VF/200)
F = 200
train_data = pca_200_train.reshape(V, F, D)

print(train_data.shape)

#train_states = None

    
VF, D = pca_200_test.shape
V = int(VF/200)
F = 200
test_data = pca_200_test.reshape(V, F, D)
print(test_data.shape)


X_train_new, X_val, y_train_new, y_val = train_test_split(train_data, y_train, test_size=0.25, random_state=42)

In [ ]:

X_train_hyp, X_val_hyp, y_train_hyp, y_val_hyp = train_test_split(X_train, y_train, test_size=0.25, random_state=42)




In [ ]:
Nf = 200
H = 160
W = 240
batch_size = 50

Nv_train = len(X_train_hyp)
Nv_val = len(X_val_hyp)
Nv_test = len(X_test)



x_train_mean = X_train_hyp.mean(axis=0)
x_train_nomean = X_train_hyp - x_train_mean
x_val_nomean = X_val_hyp - x_train_mean
x_test_nomean = X_test - x_train_mean


x_train_nomean = x_train_nomean.reshape(Nv_train*Nf, 1, H, W)
y_train_hyp = y_train_hyp.reshape(Nv_train*Nf, 1)


x_val_nomean = x_val_nomean.reshape(Nv_val*Nf, 1, H, W)
y_val_hyp = y_val_hyp.reshape(Nv_val*Nf, 1)

print(x_train_nomean.shape)
print(y_train_hyp.shape)

train_Ntrue = y_train_hyp.sum()
train_c1_w = 1.0/train_Ntrue
train_c0_w = 1.0/(len(y_train_hyp) - train_Ntrue)
train_c_wtot = train_c1_w + train_c0_w
train_c1_w /= train_c_wtot
train_c0_w /= train_c_wtot

train_c_w = (1-y_train_hyp)*train_c0_w + y_train_hyp*train_c1_w






print(train_Ntrue, train_c0_w, train_c1_w)



x_test_nomean = x_test_nomean.reshape(Nv_test*Nf, 1, H, W)
y_test = y_test.reshape(Nv_test*Nf, 1)

test_Ntrue = y_test.sum()
test_c1_w = 1.0/test_Ntrue
test_c0_w = 1.0/(len(y_test) - test_Ntrue)
test_c_wtot = test_c0_w + test_c1_w
test_c1_w /= test_c_wtot
test_c0_w /= test_c_wtot

test_c_w = (1-y_test)*test_c0_w + y_test*test_c1_w



val_Ntrue = y_val_hyp.sum()
val_c1_w = 1.0/val_Ntrue
val_c0_w = 1.0/(len(y_val_hyp) - val_Ntrue)
val_c_wtot = val_c1_w + val_c0_w
val_c1_w /= val_c_wtot
val_c0_w /= val_c_wtot

val_c_w = (1-y_val_hyp)*val_c0_w + y_val_hyp*val_c1_w








In [ ]:
x_train_expanded = np.repeat(x_train_nomean, 3, axis=1)
print(x_train_expanded.shape)




In [ ]:
trainset = TensorDataset(torch.Tensor(x_train_nomean).to(device), torch.Tensor(y_train_hyp).to(device), torch.Tensor(train_c_w).to(device))
train_dl = DataLoader(trainset, batch_size=batch_size, shuffle=True)

#testset = TensorDataset(torch.Tensor(x_test_nomean).to(device), torch.Tensor(y_test).to(device), torch.Tensor(test_c_w).to(device))
#test_dl = DataLoader(testset, batch_size=batch_size, shuffle=False)

#valset = TensorDataset(torch.Tensor(x_val_nomean).to(device), torch.Tensor(y_val_hyp).to(device), torch.Tensor(val_c_w).to(device))
#val_dl = DataLoader(valset, batch_size=batch_size, shuffle=False)

In [ ]:
class CNN(nn.Module):
    def __init__(self, hidden_size_1=1000, hidden_size_2=200):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=11, stride=4, padding=5)
        self.relu1 = nn.ReLU()
        self.conv2 = nn.Conv2d(16, 16, kernel_size=5, stride=4, padding=2)
        self.relu2 = nn.ReLU()
        self.flatten = nn.Flatten(start_dim=1)
        self.fc1 = nn.Linear(16*10*15, hidden_size_1)
        self.relu3 = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size_1, hidden_size_2)
        self.relu4 = nn.ReLU()

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.flatten(x)
        x = self.fc1(x)
        hidden = self.relu3(x)
        x = self.fc2(hidden)
        x = self.relu4(x)
        return hidden, x
    
    
    
    
torch.cuda.empty_cache()


    
    






hidden_size_1 = 1000
hidden_size_2 = 200




"""
model = nn.Sequential(
    nn.Conv2d(1, 16, 11, padding=5, stride=4),
    nn.ReLU(),
    nn.Conv2d(16, 16, 5, padding=2, stride=4),
    nn.ReLU(),
    nn.Flatten(start_dim=1),
    nn.Linear(16*10*15, 1000),
    nn.ReLU(),
    nn.Linear(1000, 1),
    nn.Sigmoid()
)
"""


"""model = nn.Sequential(
    nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2, stride=2),
    nn.Conv2d(16, 16, kernel_size=3, stride=1, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2, stride=2),
    nn.Flatten(),
    nn.Linear(16 * 40 * 60, 1000),
    nn.ReLU(),
    nn.Linear(1000, 200),
    nn.ReLU(),
    nn.Linear(200,1),
    nn.Sigmoid()
)"""

"""model = nn.Sequential(
    nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2, stride=2),
    nn.Conv2d(16, 16, kernel_size=3, stride=1, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2, stride=2),
    nn.Flatten(),
    nn.Linear(16 * 40 * 60, 1000),
    nn.ReLU(),
    nn.Linear(1000, 400),
    nn.ReLU(),
    nn.Linear(400,1),
    nn.Sigmoid()
)"""



"""model = nn.Sequential(
    nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2, stride=2),
    nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2, stride=2),
    nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2, stride=2),
    nn.Flatten(),
    nn.Linear(128 * 20 * 30, 512),
    nn.ReLU(),
    nn.Linear(512, 1),
    nn.Sigmoid()
)"""




model = nn.Sequential(
    nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
    nn.ReLU(),
    nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
    nn.ReLU(),
    nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
    nn.ReLU(),
    nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
    nn.ReLU(),
    nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1),
    nn.ReLU(),
    nn.AdaptiveAvgPool2d((1, 1)),
    nn.Flatten(),
    nn.Linear(256, 256),
    nn.ReLU(),
    nn.Linear(256, 1),
    nn.Sigmoid()
)



#hidden_size_1 = 1000
#hidden_size_2 = 200
"""model = nn.Sequential(
            nn.Flatten(start_dim=1),
            nn.Linear(240*160, hidden_size_1),
            nn.ReLU(),
            nn.Linear(hidden_size_1, hidden_size_2),
            nn.ReLU(),
            nn.Linear(hidden_size_2, 1),
            nn.Sigmoid()
)"""

model.to(device)

lr = 1e-6
weight_decay = 1e-3

#loss_criterion = nn.MSELoss() 
#loss_criterion = nn.BCELoss() 
#loss_criterion = lambda pred, true, weight: weight*
loss_criterion = F.binary_cross_entropy
optimiser = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)

Nepochs=30



tr_preds = np.zeros((len(train_dl), batch_size, 1))
tr_trues = np.zeros((len(train_dl), batch_size, 1))

#te_preds = np.zeros((len(test_dl), batch_size, 1))
#te_trues = np.zeros((len(test_dl), batch_size, 1))



for n in range(Nepochs):
    model.train()
    tr_avg_loss = 0.0
    tr_avg_acc = 0.0
    
    for i,batch in enumerate(train_dl):
        x, y_true, c_w = batch
        x = x.to(device)
        y_true = y_true.to(device)
        c_w = c_w.to(device)
        optimiser.zero_grad()
        y = model(x)
        loss = loss_criterion(y, y_true, weight=c_w)
        pred_class = (y > 0.5)
        true_class = (y_true > 0.5)
        tr_avg_acc += (pred_class == true_class).sum().item()

        tr_preds[i] = pred_class.detach().cpu().numpy()
        tr_trues[i] = true_class.detach().cpu().numpy()

        tr_avg_loss += loss.item()
        loss.backward()
        optimiser.step()
    tr_avg_loss /= len(train_dl)
    tr_avg_acc /= len(train_dl)*batch_size

    tr_prec = precision_score(tr_trues.flatten(), tr_preds.flatten(), zero_division=0)
    tr_recall = recall_score(tr_trues.flatten(), tr_preds.flatten(), zero_division=0)
    tr_f1 = f1_score(tr_trues.flatten(), tr_preds.flatten(), zero_division=0)
    tr_bal_acc = balanced_accuracy_score(tr_trues.flatten(), tr_preds.flatten())

    model.eval()
    te_avg_loss = 0.0
    te_avg_acc = 0.0


    """with torch.no_grad():
        for i, batch in enumerate(test_dl):
            x, y_true, c_w = batch
            y = model(x)
            loss = loss_criterion(y, y_true, weight=c_w)
            te_avg_loss += loss.item()
            pred_class = (y > 0.5)
            true_class = (y_true > 0.5)
            te_avg_acc += (pred_class == true_class).sum().item()
            te_preds[i] = pred_class.detach().cpu().numpy()
            te_trues[i] = true_class.detach().cpu().numpy()
    te_avg_loss /= len(test_dl)
    te_avg_acc /= len(test_dl)*batch_size
    
    te_prec = precision_score(te_trues.flatten(), te_preds.flatten(), zero_division=0)
    te_recall = recall_score(te_trues.flatten(), te_preds.flatten(), zero_division=0)
    te_f1 = f1_score(te_trues.flatten(), te_preds.flatten(), zero_division=0)
    te_bal_acc = balanced_accuracy_score(te_trues.flatten(), te_preds.flatten())"""

    #print(n, tr_avg_loss, tr_avg_acc, tr_bal_acc, te_avg_loss, te_avg_acc, tr_f1, te_f1)
    #print("Epoch, Loss, Acc, Bal Acc, F1")
    #print(n, tr_avg_loss, tr_avg_acc, tr_bal_acc, tr_f1)
    print("Epoch: {}, Train Loss: {:.4f}, Acc: {:.4f}, Bal Acc: {:.4f}, F1: {:.4f}".format(n, tr_avg_loss, tr_avg_acc, tr_bal_acc, tr_f1))







In [ ]:
feature_extractor = nn.Sequential(*list(model.children())[:-2])
num_features = 512

def extract_features(dataloader):
    dl_size = len(dataloader)
    print(dl_size)
    print(batch_size)
    
    dl_size = dl_size*batch_size
    features = np.zeros((dl_size, num_features))
    for i, batch in enumerate(dataloader):
        x, y_true, c_w = batch
        #print(x.shape)
        #y = pretrained_model(x)
        #print(y.shape)

        batch_features = feature_extractor(x)
        features[i * batch_size : (i + 1) * batch_size] = np.squeeze(batch_features.cpu().detach().numpy())
        
    print(features.shape)

    features_reshaped = features.reshape(int(dl_size/200), 200, num_features)
    return features_reshaped


#trainset = TensorDataset(torch.Tensor(x_train_nomean).to(device), torch.Tensor(y_train_hyp).to(device), torch.Tensor(train_c_w).to(device))
train_dl = DataLoader(trainset, batch_size=batch_size, shuffle=False)


esn_input_train = extract_features(train_dl)
esn_input_test = extract_features(test_dl)
esn_input_val = extract_features(val_dl)



In [ ]:
print(esn_input_train.shape)
print(esn_input_train)

y_train = y_train_hyp.reshape(42, 200, 1)
y_test = y_test.reshape(14, 200, 1)
y_val = y_val_hyp.reshape(14, 200, 1)

In [ ]:





#data_transforms = transforms.Compose([transforms.Grayscale(num_output_channels=3), transforms.ToTensor()])


pretrained_model = models.resnet50(pretrained=True)
pretrained_model.to(device)
pretrained_model.eval()

# Remove the last layer (classification layer) to extract features
feature_extractor = nn.Sequential(*list(pretrained_model.children())[:-1])

# Set the model to evaluation mode
feature_extractor.eval()

# Define your grayscale image tensor


# Forward pass to extract features
features = feature_extractor(torch.Tensor(x_train_expanded).to(device))

# Print the shape of the extracted features
print(features.shape)








In [ ]:
#vids_frames, dim = esn_input_train.shape
#vids = int(vids_frames/200)
#frames = 200
#data = esn_input_train.reshape(vids, frames, dim)




X_train_opt, X_val, y_train_opt, y_val = train_test_split(esn_input_train, y_train, test_size=0.25, random_state=42)





In [ ]:
import optuna
input_size = 512
reservoir_size = 1000
output_size = 1


#np.random.seed(75)


#def prune(trial):





def objective(trial):
    # Define the search space for hyperparameters
    #reservoir_size = trial.suggest_int("reservoir_size", 500, 3000)
    spectral_radius = trial.suggest_float("spectral_radius", 0.0, 1.0)
    sparsity = trial.suggest_float("sparsity", 0.0, 0.2)
    leaking_rate = trial.suggest_float("leaking_rate", 0.7, 0.99)
    input_scaling = trial.suggest_float("input_scaling", 0.0, 1.0)
    threshold = trial.suggest_float("threshold", -0.75, 1.75)
    ridge_alpha = trial.suggest_float("ridge_alpha", 0.00001, 1.0)
    #random_seed = trial.suggest_int("random_seed", 1, 200)

    # Initialize the ESN with the given hyperparameters
    esn = ESN(input_size=input_size, reservoir_size=reservoir_size, output_size=output_size, 
              leaking_rate=leaking_rate, spectral_radius=spectral_radius, input_scaling=input_scaling, 
              threshold=threshold, sparsity=sparsity, ridge_alpha=ridge_alpha, random_seed=83)
    
    
    #esn = ESN(input_size=input_size, reservoir_size=reservoir_size, output_size=output_size, 
    #          leaking_rate=0.95, spectral_radius=spectral_radius, input_scaling=input_scaling, sparsity=0.1, 
    #          ridge_alpha=0.5, threshold=threshold, random_seed=83)
    
    #esn = ESN(input_size=input_size, reservoir_size=reservoir_size, output_size=output_size,
    #         leaking_rate=0.95, sparsity=0.1, random_seed=random_seed)

    # Train and evaluate the ESN on the training set
    #esn.fit(X_train_opt, y_train_opt)
    #esn.fit(X_train_new, y_train_new)
    esn.fit(esn_input_train, y_train)
    
    
    y_pred, y_prob = esn.predict(esn_input_val)
    score = f1_score(y_val.flatten(), y_pred.flatten())
    #score = roc_auc_score(y_val.flatten(), y_prob.flatten())
    print("Balanced acc: ", balanced_accuracy_score(y_val.flatten(), y_pred.flatten()))
    print("F1: ", f1_score(y_val.flatten(), y_pred.flatten()))
    print("AUC: ", roc_auc_score(y_val.flatten(), y_prob.flatten()))
    
    y_prob = y_prob.flatten()
    anomaly_probs = y_prob[y_val.flatten().astype(bool)]
    non_anomaly_probs = y_prob[~y_val.flatten().astype(bool)]

    plt.hist(anomaly_probs, bins=50, alpha=0.5, label='Anomaly')
    plt.hist(non_anomaly_probs, bins=50, alpha=0.5, label='Normal')
    plt.legend(loc='upper right')
    plt.show()
    
    #trial_number = trial.number
    #if prune(trial):
    #    raise optuna.exceptions.TrialPruned()
    #print(trial.params, score)

    return score


study = optuna.create_study(direction="maximize")


study.optimize(objective, n_trials=100)#, n_jobs=-1)


best_params = study.best_params
best_score = study.best_value
print(f"Best hyperparameters: {best_params}")
print(f"Best score: {best_score}")
optuna.visualization.plot_param_importances(study)
plot_optimization_history(study)





In [ ]:
print(np.unique(y_test))

In [ ]:




#esn = ESN(input_size=400, reservoir_size=500, output_size=1, alpha=0.5, gamma=0.1, spectral_radius=0.95, sparsity=0.1)
#esn = ESN(input_scaling=0.6075448519014384, input_size=400,
    #leaking_rate=0.17052412368729153, output_size=1, reservoir_size=1000,
    #sparsity=0.06505159298527952, spectral_radius=0.9488855372533332,
    #threshold=0.9656320330745594)
    
#np.random.seed(42)
#esn = ESN(input_scaling=0.6998812876189486, input_size=400,
#    leaking_rate=0.08210799041387684, output_size=1, reservoir_size=1000,
#    sparsity=0.9252733800015754, spectral_radius=0.3364410948180354, threshold=0.39462707778742845)

#0.9219100448194322


esn = ESN(input_size=512, reservoir_size=1000, output_size=1, leaking_rate=0.804889704825926, spectral_radius=0.11146548076698023, 
          sparsity=0.02233328574171222, input_scaling=0.7663835438773781, threshold=0.15999184432185015, ridge_alpha=0.5101612068080433, random_seed=83)



#train_ESN(esn, pca_400, labels, epochs)


# ESN trained using dataset as shape (V*F, D)
#train_states = esn.fit(pca_400_train, labels_train)
#y_pred = esn.predict(pca_400_test)
#esn_accuracy = recall_score(labels_test, y_pred)
#print("ESN Accuracy: ", esn_accuracy)
#auc_score = roc_auc_score(labels_test, y_pred)
#print("ESN AUC: ", auc_score)




# Trained using (V, F, D)
#VF, D = pca_400_train.shape
#V = int(VF/200)
#F = 200
#train_data = pca_400_train.reshape(V, F, D)

#train_states = None



#esn.fit(esn_input_train, y_train)

esn.fit(esn_input_train, y_train)
    
#VF, D = pca_400_test.shape
#V = int(VF/200)
#F = 200
#test_data = pca_400_test.reshape(V, F, D)





#acc = np.zeros((V))
#for i in range(V):
#    y_pred = esn.predict(test_data[i])
#    esn_accuracy = recall_score(y_test[i], y_pred)
#    acc[i] = esn_accuracy
#    if 1 in y_test[i]:
#        auc_score = roc_auc_score(y_test[i], y_pred)
#        print(f"Accuracy for video {i}: {esn_accuracy}     AUC for video {i}: {auc_score}")
#    else:
#        print(f"Accuracy for video {i}: {esn_accuracy}")


#print(np.mean(acc))



#esn_pred, prob = esn.predict(esn_input_test)
esn_pred, prob = esn.predict(esn_input_test)
fpr, tpr, thresholds = roc_curve(y_test.flatten(), prob.flatten())
auc_score = roc_auc_score(y_test.flatten(), prob.flatten())


print("Actual anomalies: ", np.count_nonzero(y_test.flatten()))
print("Predicted anomalies", np.count_nonzero(esn_pred.flatten()))
print(prob.flatten())
print("Pred", esn_pred.shape)
print("Pred flattened", esn_pred.flatten().shape)
print("y_test", y_test.shape)
print("y_test flattened", y_test.flatten().shape)
#print((prob >= 0.5).astype(int))

#print(esn_pred.flatten())
#print(y_test.flatten())
balanced_accuracy = balanced_accuracy_score(y_test.flatten(), esn_pred.flatten())
print("Bal Acc: ", balanced_accuracy)
print("F1: ", f1_score(y_test.flatten(), esn_pred.flatten()))
print("AUC: ", auc_score)



plt.plot(fpr, tpr)
plt.title(f'ROC Curve (AUC = {auc_score:.2f})')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.show()




cm = confusion_matrix(y_test.flatten(), esn_pred.flatten())

disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot()






#esn_accuracy = accuracy_score(y_test, y_pred)
#print("ESN Accuracy: ", esn_accuracy)
#auc_score = roc_auc_score(y_test, y_pred)
#print("ESN AUC: ", auc_score)

In [ ]:
optuna.visualization.plot_parallel_coordinate(study)

In [ ]:
optuna.visualization.plot_param_importances(study)

In [ ]:
plt.scatter(range(len(prob.flatten())), prob.flatten())
plt.xlabel('Frame')
plt.ylabel('Predicted probability')
plt.show()




plt.hist(prob.flatten(), bins=10)
plt.xlabel('Predicted probability')
plt.ylabel('Count')
plt.show()




anomalies = y_test.astype(bool)
non_anomalies = ~anomalies

# plot scatter plot
plt.scatter(np.where(anomalies)[0], prob[anomalies], color='red', label='anomalies')
plt.scatter(np.where(non_anomalies)[0], prob[non_anomalies], color='blue', label='non-anomalies')
plt.xlabel('Sample Index')
plt.ylabel('Predicted Probability')
plt.legend()
plt.show()




y_prob = prob.flatten()
anomaly_probs = y_prob[y_val.flatten().astype(bool)]
non_anomaly_probs = y_prob[~y_val.flatten().astype(bool)]

plt.hist(anomaly_probs, bins=50, alpha=0.5, label='Anomaly')
plt.hist(non_anomaly_probs, bins=50, alpha=0.5, label='Normal')
plt.legend(loc='upper right')
plt.show()



